## Decoding bunch pattern data

In [1]:
from karabo_data import RunDirectory
import bunch_pattern

In [2]:
run = RunDirectory('/gpfs/exfel/exp/SCS/201802/p002197/raw/r0462/')

The bunch pattern is saved by the time server as an array of 32-bit integers, one for each possible pulse.

In [3]:
bunch_pattern_tables = run.get_array('SCS_RR_UTC/TSYS/TIMESERVER', 'bunchPatternTable.value', extra_dims=['pulse_slot'])

In [4]:
bunch_pattern_tables[:5]

<xarray.DataArray (trainId: 5, pulse_slot: 2700)>
array([[2146089,   32768,   33024, ...,       0,       0,       0],
       [2146089,   32768,   33024, ...,       0,       0,       0],
       [2146089,   32768,   33024, ...,       0,       0,       0],
       [2146089,   32768,   33024, ...,       0,       0,       0],
       [2146089,   32768,   33024, ...,       0,       0,       0]],
      dtype=uint32)
Coordinates:
  * trainId  (trainId) uint64 45297262 45297263 45297264 45297265 45297266
Dimensions without coordinates: pulse_slot

## Destinations

The 32 bits of each entry record different information, including the destination of the pulse.

E.g. we can see which pulses in each train were sent to different SASEs:

In [5]:
for table in bunch_pattern_tables[:10]:
    tid = table.trainId.item()
    sase1_pulses = bunch_pattern.indices_at_sase(table.data, sase=1)
    sase3_pulses = bunch_pattern.indices_at_sase(table.data, sase=3)
    print(f"Train {tid}: {len(sase1_pulses)} pulses to SASE1, {len(sase3_pulses)} to SASE3")

Train 45297262: 128 pulses to SASE1, 0 to SASE3
Train 45297263: 128 pulses to SASE1, 0 to SASE3
Train 45297264: 128 pulses to SASE1, 0 to SASE3
Train 45297265: 128 pulses to SASE1, 0 to SASE3
Train 45297266: 0 pulses to SASE1, 30 to SASE3
Train 45297267: 128 pulses to SASE1, 0 to SASE3
Train 45297268: 128 pulses to SASE1, 0 to SASE3
Train 45297269: 128 pulses to SASE1, 0 to SASE3
Train 45297270: 128 pulses to SASE1, 0 to SASE3
Train 45297271: 0 pulses to SASE1, 30 to SASE3


The *pulsePatternDecoder* Karabo device also produces this information:

In [6]:
print(run.get_array('SCS_RR_UTC/MDL/BUNCH_DECODER', 'sase3.nPulses.value')[:10])

<xarray.DataArray (trainId: 10)>
array([ 0,  0,  0,  0, 30,  0,  0,  0,  0, 30], dtype=int32)
Coordinates:
  * trainId  (trainId) uint64 45297262 45297263 45297264 45297265 45297266 ...


We can also check this against an *X-ray Gas Monitor* (XGM) in SASE3.
In trains where bunches are sent to SASE3, the XGM registers pulses with intensity > 3000,
compared to < 1000 for the other trains.

This XGM didn't record data for all trains, so we need to align the data.
In the output below, missing data from the XGM shows up as `nan` (Not A Number).
If we aligned with `join='inner'`, these rows would be dropped instead.

In [7]:
import xarray
sa3_xgm_intensity = run.get_array('SA3_XTD10_XGM/XGM/DOOCS:output', 'data.intensityTD')
bunch_pattern_tables, sa3_xgm_intensity = xarray.align(bunch_pattern_tables, sa3_xgm_intensity, join='outer')

for bunch_table, intensities in zip(bunch_pattern_tables[:10], sa3_xgm_intensity):
    print(f"Train {bunch_table.trainId.item()}:")
    sa3_pulses = bunch_pattern.indices_at_sase(bunch_table.values, sase=3)
    print(f"  Bunches to SASE3: {sa3_pulses[:10]}...")
    print(f"   XGM intensities: {intensities[:5].values}...")
    print()

Train 45297262:
  Bunches to SASE3: []...
   XGM intensities: [437.2743  206.8512  283.20245 418.04456 349.41214]...

Train 45297263:
  Bunches to SASE3: []...
   XGM intensities: [nan nan nan nan nan]...

Train 45297264:
  Bunches to SASE3: []...
   XGM intensities: [121.54636  72.26654 175.06305  99.43744  72.64107]...

Train 45297265:
  Bunches to SASE3: []...
   XGM intensities: [115.02722   60.6854   110.661606 102.08841  128.99011 ]...

Train 45297266:
  Bunches to SASE3: [240 256 272 288 304 320 336 352 368 384]...
   XGM intensities: [3.4325640e+03 2.0774655e+00 6.4372176e-01 3.7218819e+00 3.2275093e+03]...

Train 45297267:
  Bunches to SASE3: []...
   XGM intensities: [124.928825 174.05649  171.65717  113.17213  132.65349 ]...

Train 45297268:
  Bunches to SASE3: []...
   XGM intensities: [nan nan nan nan nan]...

Train 45297269:
  Bunches to SASE3: []...
   XGM intensities: [123.30196   90.413635  91.29729   89.01501   65.168045]...

Train 45297270:
  Bunches to SASE3: []...


## Pulse charge

You can also get the pulse charge in nanocoulombs (nC):

In [8]:
bunch_pattern.get_charge(bunch_pattern_tables[0, :200])

array([0.36, 0.  , 0.  , 0.  , 0.36, 0.  , 0.  , 0.  , 0.36, 0.  , 0.  ,
       0.  , 0.36, 0.  , 0.  , 0.  , 0.36, 0.  , 0.  , 0.  , 0.36, 0.  ,
       0.  , 0.  , 0.36, 0.  , 0.  , 0.  , 0.36, 0.  , 0.  , 0.  , 0.36,
       0.  , 0.  , 0.  , 0.36, 0.  , 0.  , 0.  , 0.36, 0.  , 0.  , 0.  ,
       0.36, 0.  , 0.  , 0.  , 0.36, 0.  , 0.  , 0.  , 0.36, 0.  , 0.  ,
       0.  , 0.36, 0.  , 0.  , 0.  , 0.36, 0.  , 0.  , 0.  , 0.36, 0.  ,
       0.  , 0.  , 0.36, 0.  , 0.  , 0.  , 0.36, 0.  , 0.  , 0.  , 0.36,
       0.  , 0.  , 0.  , 0.36, 0.  , 0.  , 0.  , 0.36, 0.  , 0.  , 0.  ,
       0.36, 0.  , 0.  , 0.  , 0.36, 0.  , 0.  , 0.  , 0.36, 0.  , 0.  ,
       0.  , 0.36, 0.  , 0.  , 0.  , 0.36, 0.  , 0.  , 0.  , 0.36, 0.  ,
       0.  , 0.  , 0.36, 0.  , 0.  , 0.  , 0.36, 0.  , 0.  , 0.  , 0.36,
       0.  , 0.  , 0.  , 0.36, 0.  , 0.  , 0.  , 0.36, 0.  , 0.  , 0.  ,
       0.36, 0.  , 0.  , 0.  , 0.36, 0.  , 0.  , 0.  , 0.36, 0.  , 0.  ,
       0.  , 0.36, 0.  , 0.  , 0.  , 0.36, 0.  , 0.